# Python Brasil 2021

## Seleção de portfólios com Algoritmo Genético e Rede Neural Artificial LSTM

Esta primeira célula é responsável por atualizar algumas bibliotecas para que seja possível extrair os dados de cotações de ações do yahoo finance


In [ ]:
! pip install --upgrade numpy
! pip install --upgrade pandas
! pip install --upgrade pandas-datareader

# reiniciar o kernel após executar esta célula (ctrl + M)

É necessário montar o Google Drive, para acessar a pasta "pybr21" em que deverão estar os arquivos do repositório e importar os arquivos implementados para o algoritmo genético (ag_py)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [2]:
import sys
sys.path.append('/content/drive/MyDrive/pybr21')

from ag_py.aux import *
from ag_py.globais import *
from ag_py.grafico import *
from ag_py.classes import *
from ag_py.metricas import *
from nsga2 import *
from ag_py.operadores import *

import os
os.chdir('/content/drive/MyDrive/pybr21')

In [3]:
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import datetime 

In [ ]:
ativos_label = ["MGLU3.SA", "VALE3.SA", "ABEV3.SA", "B3SA3.SA","BBSE3.SA", "BRML3.SA", "BBDC3.SA", "BBDC4.SA","BRAP4.SA", "CMIG4.SA","CIEL3.SA","COGN3.SA"]
#ativos_label = ["ABEV3.SA", "B3SA3.SA","BBSE3.SA", "BRML3.SA", "BBDC3.SA", "BBDC4.SA","BRAP4.SA","BBAS3.SA","BRKM5.SA","BRFS3.SA", "CCRO3.SA","CMIG4.SA","CIEL3.SA","COGN3.SA","CSAN3.SA","CVCB3.SA","CYRE3.SA","ECOR3.SA","ELET3.SA","ELET6.SA","EMBR3.SA","ENBR3.SA","EGIE3.SA","EQTL3.SA","FLRY3.SA","GGBR4.SA","GOAU4.SA","GOLL4.SA","HYPE3.SA","IGTA3.SA","ITSA4.SA","ITUB4.SA","JBSS3.SA","KLBN11.SA","RENT3.SA","LAME4.SA","LREN3.SA","MGLU3.SA","MRFG3.SA","MRVE3.SA","MULT3.SA","PETR3.SA","PETR4.SA","QUAL3.SA","RADL3.SA","SBSP3.SA","SANB11.SA","CSNA3.SA","SULA11.SA","TAEE11.SA","VIVT3.SA","TOTS3.SA","UGPA3.SA","USIM5.SA","VALE3.SA","WEGE3.SA","YDUQ3.SA","^BVSP"]

ativos_cotacoes = []
for ativo_label in ativos_label:
  print("taking " + ativo_label + " data")
  ativos_cotacoes.append(((ativo_label),(web.DataReader(ativo_label,start="2015-01-01",end="2019-12-12", data_source='yahoo'))))


A célula seguinte é responsável pela utilização de um modelo de Aprendizado de Máquina para fazer a predição dos preços, diariamente, das ações. Para entender melhor como funciona, recomendo assistir o vídeo do mini curso sobre o código ([primeiro link do README.md deste repositório](https://github.com/paulacampigotto/mini-curso_ML_SEI))


In [ ]:
taxas_de_retorno = []
for ativo in ativos_cotacoes:

  cotacoes_df = ativo[1].filter(['Close'])
  cotacoes = cotacoes_df.values

  normalizador = MinMaxScaler(feature_range=(0, 1)) 
  cotacoes_normalizadas = normalizador.fit_transform(cotacoes)

  dias_treinamento = math.ceil( len(cotacoes) *.8) 
  cotacoes_treinamento = cotacoes_normalizadas[0:dias_treinamento]


  x_treino=[]
  y_treino = []
  for i in range(30,len(cotacoes_treinamento)):
      x_treino.append(cotacoes_treinamento[i-30:i,0])
      y_treino.append(cotacoes_treinamento[i,0])


  x_treino, y_treino = np.array(x_treino), np.array(y_treino)
  x_treino = np.reshape(x_treino, (x_treino.shape[0],x_treino.shape[1],1))


  model = Sequential()
  model.add(LSTM(units=50, return_sequences=True,input_shape=(x_treino.shape[1],1)))
  model.add(LSTM(units=50, return_sequences=False))
  model.add(Dense(units=25))
  model.add(Dense(units=1))

  model.compile(optimizer='adam', loss='mean_squared_error')


  model.fit(x_treino, y_treino, batch_size=1, epochs=1)


  cotacoes_teste = cotacoes_normalizadas[dias_treinamento - 30:]

  x_teste = []
  y_teste =  cotacoes[dias_treinamento :] 
  for i in range(30,len(cotacoes_teste)):
      x_teste.append(cotacoes_teste[i-30:i,0])
 
  x_teste = np.array(x_teste)
  x_teste = np.reshape(x_teste, (x_teste.shape[0],x_teste.shape[1],1))


  predicoes = model.predict(x_teste) 
  predicoes = normalizador.inverse_transform(predicoes)


  rmse=np.sqrt(np.mean(((predicoes- y_teste)**2)))
  print(rmse)

  treino = cotacoes_df[:dias_treinamento]
  valido = cotacoes_df[dias_treinamento:]
  valido['predicoes'] = predicoes
  plt.figure(figsize=(16,8))
  plt.title('Ativo ' + ativo[0])
  plt.xlabel('Data')
  plt.ylabel('Preço de fechamento (R$)')
  plt.plot(treino['Close'], color = 'black')
  plt.plot(valido['Close'], color = 'green')
  plt.plot(valido['predicoes'], color = 'red')
  plt.legend(['Histórico', 'Valor real', 'Predição'])
  plt.savefig('graficos/' + ativo[0] + '.png')


  if(taxas_de_retorno = []):
    for l in range(len(valido)-1):
      taxas_de_retorno.append([])

  for j in range(len(valido)-1):
      taxas_de_retorno[j].append((ativo[0],(valido['predicoes'][j+1]/valido['Close'][j])))




Por fim, salvam-se em um arquivo as 9 ações selecionadas em cada dia

In [13]:
best = []


for line in range(len(taxas_de_retorno)):
  taxas_de_retorno[line].sort(key=lambda x: x[1], reverse = True)
  best.append([])
  for x in range(9):
    best[line].append(taxas_de_retorno[line][x][0])
  with open('arquivos/selected.txt', 'a+') as f:
    f.write(str(best[line]) + '\n')



Chamada de otimização do algoritmo genético (acessa o arquivo selected.txt)

In [ ]:
otimizar_carteira()